# CÁLCULO DE VIGA PROTENDIDA - V1021

**OBRA:** Maltaria Campos Gerais

**Cliente:** Agrária/Viero

**Data:** 14/11/2022

**Cálculo:** Gabriel Garcia

<img src="img/logoRS.jpg" alt="RS Engenharia & Arquitetura" width="200px"/>

Todas as grandezas apresentadas possuem como unidades base kN [força] e cm [comprimento].

In [1]:
import handcalcs.render
import pandas as pd
import numpy as np
from math import floor,ceil,sqrt,exp
from numpy import log
from ggcp import calcpsi, linha_neutra_cp_ret
    
pi = 3.141592653

# Informações Iniciais:

## Geometria da Viga:

In [2]:
%%render
# Geometria da Viga:

h1 = 100
h0 = 84
b = 45

A_c0 = b*h0
A_c1 = b*h1

I_c0 = (b*h0**3/12)
I_c1 = (b*h1**3/12)

y_0 = h0/2
y_1 = h1/2

W_0 = I_c0/(y_0)
W_1 = I_c1/(y_1)

Vao = 1330

## Número de seções para dividir o vão:
# Escolher número impar
nS = 31

<IPython.core.display.Latex object>

## Propriedades dos Materiais:

In [3]:
%%render
# Propriedades dos Materiais:

f_ck = 4
f_ctm = 0.03*(10*f_ck)**(2/3)
f_ptk = 190
f_pyd = 148.7
f_yk = 50
f_cj = 2.8

E_s = 21000
E_p = 20000

<IPython.core.display.Latex object>

In [4]:
%%render 2 short

f_c = f_ck*0.85/1.4
E_ci = 560*(f_ck*10)**(1/2)
E_cs = E_ci*(0.8+0.2*f_ck/8)

E_cit0 = (f_cj/f_ck)**(1/2)*E_ci
E_cst0 = E_cit0*(0.8+0.2*f_cj/8)

f_yd = f_yk/1.15
sigma_p0 = 0.8*f_ptk

<IPython.core.display.Latex object>

## Carregamentos:

In [5]:
%%render 4 short

Q_t0 = 0.000025*A_c0
Q_tF = 56.64/100
Q_els = 41.38/100

<IPython.core.display.Latex object>

## Esforços no Centro do Vão:

In [6]:
%%render 2 short
M_F = (Q_tF)*Vao**2*0.125
M_0 = (Q_t0)*Vao**2*0.125
V_F = (Q_tF)*Vao/2

<IPython.core.display.Latex object>

# Deteminação da Força de Protensão:

## Limites de Tensão:

In [7]:
%%render short

sigma_ct0 = -1.2*0.03*(f_cj*10)**(2/3)
sigma_cc0 = 0.7*f_cj
sigma_ctF = -1.2*0.03*(f_ck*10)**(2/3)
sigma_ccF = 0.5*f_ck

r_0 = 0.95
r_F = 0.80

<IPython.core.display.Latex object>

## Cálculo da Força de Protensão:

In [8]:
%%render short 2

e_p0 = 32
e_p1 = 40

#tempo inicial:
P_1 = (sigma_cc0+M_0/W_0)/(r_0*(1/A_c0 + e_p0/W_0)) 
P_2 = (sigma_ct0-M_0/W_0)/(r_0*(1/A_c0 - e_p0/W_0)) 

#tempo final:
P_3 = (sigma_ctF+M_F/W_1)/(r_F*(1/A_c1 + e_p1/W_1))
P_4 = (sigma_ccF-M_F/W_1)/(r_F*(1/A_c1 - e_p1/W_1))

##Adotar valores entre:
P_min = max(P_3, P_4)
P_max = min(P_1, P_2)


<IPython.core.display.Latex object>

## Escolha da Protensão:

In [9]:
%%render 2
# Força de Protensão Adotada:

P = P_max
phi_1 = 1.52
A_p1 = 1.43

## Número de cordoalhas para adotar:
n = floor(P/(A_p1*sigma_p0))

<IPython.core.display.Latex object>

## Ancoragem das Cordoalhas:

### Comprimento de Ancoragem Básico:

In [10]:
%%render short
f_pyd = 0.9* f_ptk / 1.15

eta_p1 = 1.2

# Valores de Eta_p1:
#     
#     1.0 para fios lisos
#     1.2 para cordoalhas de 3 e 7 fios
#     1.5 para fios dentados

eta_p2 = 1.0

# Valores de Eta_p2:
#     
#     1.0 para situação de boa aderência
#     0.7 para situação de má aderência

f_ctd = (0.7*0.03*(f_cj*10)**(2/3))/1.4
f_bpd = eta_p1 * eta_p2 * f_ctd

l_bp = (7*phi_1)/36 * f_pyd/f_bpd

<IPython.core.display.Latex object>

### Cálculo do comprimento de Transferência:

In [11]:
%%render short

## para fios lisos ou dentados:
l_bpt1 = 0.7 * l_bp * sigma_p0/f_pyd

## para cordoalhas de 3 e 7 fios:
l_bpt2 = 0.5 * l_bp * sigma_p0/f_pyd

## para transferência não gradual, os valores devem ser multiplicados por 1.25
## valor adotado:

l_bpt = 1.0 * l_bpt2

l_p = (h1**2 + (0.6*l_bpt)**2)**(0.5)


<IPython.core.display.Latex object>

#  Esforços nas Seções:

## Momento Fletor e Cortante:

In [12]:
esforcos = []
esforcos_t0 = []
for i in range(3):
    esforcos.append( [0]*(nS) )
    esforcos_t0.append( [0]*(nS) )
    
# primeiro vetor contem X (cm)
# segundo vetor contem Momentos (kNcm)
# terceiro vetor contém cortante (kN)

for i in range(nS):
    esforcos[0][i] = round(i*(Vao/(nS-1)),1)
    esforcos[1][i] = round( 0.5*Q_tF*esforcos[0][i]*(Vao-esforcos[0][i]) ,1)
    esforcos[2][i] = round( 0.5*Q_tF*(Vao-2*esforcos[0][i]), 1 )
    esforcos_t0[0][i] = round(i*(Vao/(nS-1)),1)
    esforcos_t0[1][i] = round( 0.5*Q_t0*esforcos[0][i]*(Vao-esforcos[0][i]) ,1)
    esforcos_t0[2][i] = round( 0.5*Q_t0*(Vao-2*esforcos[0][i]), 1 )
    
    
tabela = pd.DataFrame(esforcos)
tabela = tabela.transpose()
tabela.rename(columns={0:'Pos (cm)',1:'M (kNcm)',2:'V (kN)'},inplace = True)
display(tabela)

,Pos (cm),M (kNcm),V (kN)
0,0.0,0.0,376.7
1,44.3,16130.1,351.6
2,88.7,31181.3,326.4
3,133.0,45085.7,301.3
4,177.3,57878.6,276.2
5,221.7,69585.1,251.1
6,266.0,80152.4,226.0
7,310.3,89608.1,200.9
8,354.7,97969.9,175.8
9,399.0,105200.0,150.7


# Escalonamento da Protensão:

## Limites de Tensão na Seção S1:

São testados os limites de tensão para t0:

Limitação pela borda superior:

In [13]:
%%render short

M_s1 = round( 0.5*Q_t0*esforcos[0][1]*(Vao-esforcos[0][1]) ,1) # Momento de peso próprio na seção s1
P_1 = (sigma_cc0+M_s1/W_0)/(r_0*(1/A_c0 + e_p0/W_0))
P_2 = (sigma_ct0-M_s1/W_0)/(r_0*(1/A_c0 - e_p0/W_0))


<IPython.core.display.Latex object>

In [14]:
%%render 1

P_s1 = min(P_1 , P_2)

n_s1 = floor(P_s1/(A_p1*sigma_p0))


<IPython.core.display.Latex object>

## Escolha do Número de Cordoalhas:

In [15]:
%%render

# Vetor com o número de cordoalhas em cada ponto de escalonamento:

qntCordoalhas = [4,6]

# Vetor com a excentricidade da protensão em cada ponto de escalonamento:

e_p_i = [40,40]

<IPython.core.display.Latex object>

# Esforços Devidos À Protensão:

In [16]:
A_p = []

for i in range(len(qntCordoalhas)):
    A_p.append(qntCordoalhas[i] * A_p1)

W_cp = []

for i in range(len(qntCordoalhas)):
    W_cp.append(round(I_c1/e_p_i[i],0))
    
alpha_p0 = E_p / E_cst0

K_p0 = []
for i in range(len(qntCordoalhas)):
    K_p0.append(round(alpha_p0*A_p[i]*(1/A_c1+e_p_i[i]/W_cp[i]),4))
    
sigma_p = []
for i in range(len(qntCordoalhas)):
    sigma_p.append(round(sigma_p0/(1+K_p0[i]),2))
    
N_p = []
for i in range(len(qntCordoalhas)):
    N_p.append(round(-sigma_p[i]*A_p[i],1))
    
M_p = []
for i in range(len(qntCordoalhas)):
    M_p.append(round(N_p[i]*e_p_i[i],1))
    
tabela1 = pd.DataFrame([A_p, W_cp, K_p0, sigma_p, N_p, M_p])
tabela1 = tabela1.transpose()
tabela1.rename(columns={0:'Ap',1:'Wcp',2:'Kp0',3:'Sigma,p',4:'Np',5:'Mp'},inplace = True)
display(tabela1)

,Ap,Wcp,Kp0,"Sigma,p",Np,Mp
0,5.72,93750.0,0.0288,147.74,-845.1,-33804.0
1,8.58,93750.0,0.0432,145.71,-1250.2,-50008.0


# Escalonamento da Força de Protensão:

In [17]:
M_g = []
x1 = []
x2 = []
x1_anc = []
x2_anc = []

for i in range(len(qntCordoalhas)):
    M_g.append(round( W_0*( (N_p[i]/A_c0) -(M_p[i]/W_0) + sigma_ct0),2) )
    #print("M_g["+str(i)+"]:",M_g[i])
        
for i in range(len(qntCordoalhas)):
    # O erro acontece no caso da raiz de delta retornar um número complexo
    try:
        c = 2*M_g[i]/Q_t0
        delta = (-Vao)**2 - 4*c
        
        #print("Delta "+str(i)+":",round(delta,2))
        x1.append(round( (Vao-(delta**0.5))/2  ,2))
        x2.append(round( (Vao+(delta**0.5))/2  ,2))
    except:
        print("##   ATENÇÃO!   ##")
        raise Exception("ERRO: ESCALONAR MELHOR A PROTENSÃO!")    

for i in range(len(qntCordoalhas)):
    x1_anc.append(x1[i]-round(l_bpt,2))
    x2_anc.append(x2[i]+round(l_bpt,2))
    if x1_anc[i] < 0:
        x1_anc[i] = 0
    if x2_anc[i] > Vao:
        x2_anc[i] = Vao
    
tabela2 = pd.DataFrame([M_g, x1,x2, x1_anc,x2_anc])
tabela2 = tabela2.transpose()
tabela2.rename(columns={0:'Mg',1:'x1',2:'x2',3:'x1-lbpt',4:'x2+lbpt'},inplace = True)
display(tabela2)

,Mg,x1,x2,x1-lbpt,x2+lbpt
0,4405.73,74.25,1255.75,0.00,1330.00
1,14938.33,309.94,1020.06,174.61,1155.39


# Perdas Progressivas da Força de Protensão:

## Fluência do Concreto:

### Fluência Rápida Irreversível

In [18]:
%%render params

# VER ITEM 12.3.3 da NBR6118:2014
# Valores para adotar:
# s = 0.38 para concreto de cimento CP-III e CP-IV
# s = 0.25 para concreto de cimento CP-I e CP-II
# s = 0.20 para concreto de cimento CP-V
s = 0.20
t_0 = 10
t_F = 18250


beta_0 = exp(s*(1-(28/t_0)**0.5))
beta_F = exp(s*(1-(28/t_F)**0.5))

phi_a = 0.8*(1 - (beta_0/beta_F))

<IPython.core.display.Latex object>

In [19]:
if s == 0.38:
    print("Adotado s = 0.38 para concreto de cimento CP-III ou CP-IV")
elif s == 0.25:
    print("Adotado s = 0.25 para concreto de cimento CP-I ou CP-II")
else:
    print("Adotado s = 0.20 para concreto de cimento CP-V (ARI) ")

Adotado s = 0.20 para concreto de cimento CP-V (ARI) 


### Fluência Lenta Irreversível:

In [20]:
%%render params 2

U = 55 #umidade média do ar

phi_c1 = 4.45 - 0.035*U # para abatimentos entre 5cm e 9cm

gamma = 1 + exp(-7.8 + 0.1 * U)

u_ar = ( 2*h1 + 2*b )

h_fic = gamma*(2*A_c1)/u_ar

phi_c2 = (42+h_fic)/(20+h_fic)

phi_fF = phi_c1*phi_c2

h_fm = h_fic/100 # espessura fictícia em metros

A_beta_f = 42*h_fm**3 - 350*h_fm**2 + 588*h_fm + 113
B_beta_f = 768*h_fm**3 - 3060*h_fm**2 + 3234*h_fm - 23
C_beta_f = -200*h_fm**3 + 13*h_fm**2 + 1090*h_fm + 183
D_beta_f = 7579*h_fm**3 - 31916*h_fm**2 + 35343*h_fm + 1931

beta_f_t0 = (t_0**2 + A_beta_f*t_0 + B_beta_f )/(t_0**2 + C_beta_f*t_0 + D_beta_f)
beta_f_tF = (t_F**2 + A_beta_f*t_F + B_beta_f )/(t_F**2 + C_beta_f*t_F + D_beta_f) 

<IPython.core.display.Latex object>

### Fluência Lenta Reversível:

In [21]:
%%render params

phi_dF = 0.4

beta_d = (t_F-t_0+20)/(t_F-t_0+70)

<IPython.core.display.Latex object>

### Fluência Total:

In [22]:
%%render 2 params
phi_t = phi_a + phi_fF*(beta_f_tF - beta_f_t0) + phi_dF * beta_d

<IPython.core.display.Latex object>

### Retração do Concreto:

In [23]:
%%render 2 params

epsilon_1s = (-8.09 + U/15 - U**2/2284 + U**3/133765 - U**4/7608150)*10**-4

epsilon_2s = (33+2*h_fic)/(20.8 + 3*h_fic)

A_beta_s = 40  
B_beta_s = 116*h_fm**3 - 282*h_fm**2 + 220*h_fm - 4.8  
C_beta_s = 2.5*h_fm**3 - 8.8*h_fm + 40.7  
D_beta_s = -75*h_fm**3 + 585*h_fm**2 + 496*h_fm - 6.8  
E_beta_s = -169*h_fm**4 + 88*h_fm**3 + 584*h_fm**2 - 39*h_fm + 0.8  

beta_s_t0 = ((t_0/100)**3 + A_beta_s*(t_0/100)**2 +  B_beta_s*(t_0/100))/((t_0/100)**3 + C_beta_s*(t_0/100)**2 +  D_beta_s*(t_0/100) + E_beta_s)

beta_s_tF = ((t_F/100)**3 + A_beta_s*(t_F/100)**2 +  B_beta_s*(t_F/100))/((t_F/100)**3 + C_beta_s*(t_F/100)**2 +  D_beta_s*(t_F/100) + E_beta_s)

epsilon_cs = epsilon_1s*epsilon_2s*(beta_s_tF - beta_s_t0)


<IPython.core.display.Latex object>

## Relaxação do Aço de Protensão:

In [24]:
relax = []
for i in range(7):
    relax.append([0]*nS)

for i in range (nS):
    relax[0][i] = int(i+1)
    relax[1][i] = esforcos[0][i] #posição x da seção
    for j in range(len(qntCordoalhas)-1,-1,-1):
        if(relax[1][i] >= x1_anc[j] and relax[1][i] <= x2_anc[j]):
            relax[2][i] = round(sigma_p[j],2)
            break
    relax[3][i] = round(relax[2][i]/f_ptk,3)
    relax[4][i] = round(calcpsi(relax[3][i]),3)
    relax[5][i] = round(relax[4][i]*2.5,3)
    relax[6][i] = round(-log(1-relax[5][i]),3)
    
    
tabela3 = pd.DataFrame(relax)
tabela3 = tabela3.transpose()
tabela3.rename(columns={0:'Seção',1:'Pos',2:'Sigma,p',3:'Sigma,p/fptk',4:'\u03A81000',5:'\u03A8,inf',6:'X,inf'},inplace = True)
display(tabela3)

,Seção,Pos,"Sigma,p","Sigma,p/fptk",Ψ1000,"Ψ,inf","X,inf"
0,1.0,0.0,147.74,0.778,0.033,0.083,0.087
1,2.0,44.3,147.74,0.778,0.033,0.083,0.087
2,3.0,88.7,147.74,0.778,0.033,0.083,0.087
3,4.0,133.0,147.74,0.778,0.033,0.083,0.087
4,5.0,177.3,145.71,0.767,0.032,0.080,0.083
5,6.0,221.7,145.71,0.767,0.032,0.080,0.083
6,7.0,266.0,145.71,0.767,0.032,0.080,0.083
7,8.0,310.3,145.71,0.767,0.032,0.080,0.083
8,9.0,354.7,145.71,0.767,0.032,0.080,0.083
9,10.0,399.0,145.71,0.767,0.032,0.080,0.083


## Cálculo das Perdas Progressivas:

In [25]:
progressivas = []

for i in range(4):
    progressivas.append([0] * nS)
    
alpha_p = E_p/E_cs

delta_Np_max = 0
delta_Mp_max = 0
    
for i in range(nS):
    pos_i = relax[1][i]
    momento = 0.5*Q_t0*pos_i*(Vao-pos_i)
    #momento = esforcos[1][i]
    #print("Momento seção "+str(i)+": "+str(int(momento)))
    for j in range(len(qntCordoalhas)-1,-1,-1):
        if(relax[1][i] >= x1_anc[j] and relax[1][i] <= x2_anc[j]):
            sigma_pi = sigma_p[j]
            Npi = N_p[j]
            Mpi = M_p[j]
            Wcpi = W_cp[j]
            e_pp = e_p_i[j]
            A_pp = A_p[j]
            break
        else:
            sigma_pi = 0
            Npi = 0
            Mpi = 0
            Wcpi = 0
            e_pp = 0
            A_pp = 0
            
    if (Npi != 0):
        progressivas[0][i] = round(Npi/A_c1 + Mpi/Wcpi + momento/Wcpi,2)
        K_p = alpha_p*A_pp*(1/A_c1 + e_pp/Wcpi)
        progressivas[1][i] = round( (( progressivas[0][i]*phi_t/E_cs + epsilon_cs - sigma_pi/E_p*relax[6][i] ) / (1 + relax[6][i] + K_p*(1+0.5*phi_t)) )*E_p,2)
        progressivas[2][i] = -round(progressivas[1][i]*A_pp  ,2)
        progressivas[3][i] = round(progressivas[2][i]*e_pp ,1)
        if(progressivas[2][i] > delta_Np_max):
            delta_Np_max = progressivas[2][i]
            delta_Mp_max = progressivas[3][i]
    else:
        progressivas[0][i] = 0
        progressivas[1][i] = 0
        progressivas[2][i] = 0
        progressivas[3][i] = 0
    
    

tabela4 = pd.DataFrame(progressivas)
tabela4 = tabela4.transpose()
tabela4.rename(columns={0:'Sigma,c,t0',1:'\u0394Sigma,p',2:'\u0394Np',3:'\u0394Mp'},inplace = True)
display(tabela4)

,"Sigma,c,t0","ΔSigma,p",ΔNp,ΔMp
0,-0.55,-28.92,165.42,6616.8
1,-0.52,-28.38,162.33,6493.2
2,-0.49,-27.83,159.19,6367.6
3,-0.47,-27.47,157.13,6285.2
4,-0.71,-30.46,261.35,10454.0
5,-0.69,-30.10,258.26,10330.4
6,-0.67,-29.75,255.26,10210.4
7,-0.65,-29.39,252.17,10086.8
8,-0.64,-29.21,250.62,10024.8
9,-0.62,-28.86,247.62,9904.8


# Tensões Normais nas Bordas:

## Bordas Superiores:

In [26]:
tensaosup = []

for i in range(3):
    tensaosup.append([0]*nS)
    
for i in range(nS):
    tensaosup[0][i] = relax[1][i]
    for j in range(len(qntCordoalhas)-1,-1,-1):
        if(relax[1][i] >= x1_anc[j] and relax[1][i] <= x2_anc[j]):
            sigma_pi = sigma_p[j]
            Npi = N_p[j]
            Mpi = M_p[j]
            Wcpi = W_cp[j]
            e_pp = e_p_i[j]
            A_pp = A_p[j]
            break
    Npx = Npi + progressivas[2][i]
    Mpx = Mpi + progressivas[3][i]
    tensaosup[1][i] = round( -abs(Npx/A_c1) + abs(Mpx/W_1) - abs(esforcos[1][i]/W_1) ,3)
    if tensaosup[1][i] > -sigma_ccF:
        tensaosup[2][i] = "OK"
    else:
        tensaosup[2][i] = "--> NAO OK <--"

tabela5 = pd.DataFrame(tensaosup)
tabela5 = tabela5.transpose()
tabela5.rename(columns={0:'Pos (cm)',1:'Sigma,s',2:'OK'},inplace = True)
display(tabela5)

,Pos (cm),"Sigma,s",OK
0,0.0,0.211,OK
1,44.3,-0.003,OK
2,88.7,-0.202,OK
3,133.0,-0.387,OK
4,177.3,-0.464,OK
5,221.7,-0.619,OK
6,266.0,-0.759,OK
7,310.3,-0.884,OK
8,354.7,-0.995,OK
9,399.0,-1.091,OK


## Bordas Inferiores:

In [27]:
tensaoinf = []

for i in range(3):
    tensaoinf.append([0]*nS)
    
for i in range(nS):
    tensaoinf[0][i] = relax[1][i]
    for j in range(len(qntCordoalhas)-1,-1,-1):
        if(relax[1][i] >= x1_anc[j] and relax[1][i] <= x2_anc[j]):
            sigma_pi = sigma_p[j]
            Npi = N_p[j]
            Mpi = M_p[j]
            Wcpi = W_cp[j]
            e_pp = e_p_i[j]
            A_pp = A_p[j]
            break
    Npx = Npi + progressivas[2][i]
    Mpx = Mpi + progressivas[3][i]
    tensaoinf[1][i] = round( -abs(Npx/A_c1) - abs(Mpx/W_1) + abs(esforcos[1][i]/W_1) ,3)
    if tensaoinf[1][i] < -sigma_ctF:
        tensaoinf[2][i] = "OK"
    else:
        tensaoinf[2][i] = "--> NAO OK <--"
    
tabela6 = pd.DataFrame(tensaoinf)
tabela6 = tabela6.transpose()
tabela6.rename(columns={0:'Pos (cm)',1:'Sigma,i',2:'OK'},inplace = True)
display(tabela6)

,Pos (cm),"Sigma,i",OK
0,0.0,-0.514,OK
1,44.3,-0.301,OK
2,88.7,-0.102,OK
3,133.0,0.081,OK
4,177.3,0.025,OK
5,221.7,0.178,OK
6,266.0,0.317,OK
7,310.3,0.441,--> NAO OK <--
8,354.7,0.551,--> NAO OK <--
9,399.0,0.645,--> NAO OK <--


# Armaduras Passivas:

## Armadura da Cunha de Tração:

In [28]:
sigma_cs0 = -10000
pos = -1
for i in range (0,nS):
    if sigma_cs0 < tensaosup[1][i]:
        sigma_cs0 = tensaosup[1][i]
        pos = i

sigma_ci0 = tensaoinf[1][pos]

print("Seção crítica de cálculo: seção",pos)

Seção crítica de cálculo: seção 0


In [29]:
%%render 2

x = sigma_cs0/(sigma_cs0 - sigma_ci0) * h0

S_x = b*x**2/2

R_ct = sigma_cs0/x*S_x

A_st = R_ct/25.00

phi_st = 1.6

N_st = ceil(A_st/((phi_st/2)**2*pi))

<IPython.core.display.Latex object>

## Pré-Alongamento de Leito:

In [30]:
prealong = []

for i in range(5):
    prealong.append([0] * nS)
    
for i in range(nS):
    for j in range(len(qntCordoalhas)-1,-1,-1):
        if(relax[1][i] >= x1_anc[j] and relax[1][i] <= x2_anc[j]):
                sigma_pw = sigma_p[j]
                Npw = N_p[j]
                Mpw = M_p[j]
                Wcpw = W_cp[j]
                break
    prealong[0][i] = Npw + progressivas[2][i]
    prealong[1][i] = Mpw + progressivas[3][i]
    prealong[2][i] = round(prealong[0][i]/A_c1 + prealong[1][i]/Wcpw ,2)
    prealong[3][i] = round( sigma_pw + progressivas[1][i] - alpha_p*prealong[2][i] ,2)
    prealong[4][i] = round(prealong[3][i]/E_p,5)

tabela7 = pd.DataFrame(prealong)
tabela7 = tabela7.transpose()
tabela7.rename(columns={0:'Np,F',1:'Mp,F',2:'Sigma,cpF', 3:'Sigma,PN',4:'e,pn'},inplace = True)
display(tabela7)

,"Np,F","Mp,F","Sigma,cpF","Sigma,PN","e,pn"
0,-679.68,-27187.2,-0.44,121.58,0.00608
1,-682.77,-27310.8,-0.44,122.12,0.00611
2,-685.91,-27436.4,-0.45,122.73,0.00614
3,-687.97,-27518.8,-0.45,123.09,0.00615
4,-988.85,-39554.0,-0.64,119.27,0.00596
5,-991.94,-39677.6,-0.64,119.63,0.00598
6,-994.94,-39797.6,-0.65,120.04,0.00600
7,-998.03,-39921.2,-0.65,120.40,0.00602
8,-999.58,-39983.2,-0.65,120.58,0.00603
9,-1002.58,-40103.2,-0.65,120.93,0.00605


## Armadura Passiva de Flexão:

![formulario](img/formulario.png)

In [31]:
d_p = [] # braço de alavanca da armadura ativa
d_s = [] # braço de alavanca da armadura passiva

for i in range(len(qntCordoalhas)):
    d_p.append(e_p_i[i] + h1/2)
    d_s.append(h1-5) # estimativa para ds
    
A_s = []
for i in range(8):
    A_s.append([0] * nS)
    
for i in range(nS):
    for j in range(len(qntCordoalhas)-1,-1,-1):
        if(relax[1][i] >= x1_anc[j] and relax[1][i] <= x2_anc[j]):
            sigma_pm = sigma_p[j]
            Npm = N_p[j]
            Mpm = M_p[j]
            Wcpm = W_cp[j]
            A_pm = A_p[j]
            d_sm = d_s[j]
            d_pm = d_p[j]
            break
    Md = esforcos[1][i]*1.4
    retorno = linha_neutra_cp_ret(b,h1,f_c,A_pm,prealong[4][i],d_pm,d_sm,Md)
    A_s[0][i] = int(Md)
    A_s[1][i] = retorno[0]
    A_s[2][i] = round(retorno[0]/d_sm,3)
    A_s[3][i] = retorno[2]
    A_s[5][i] = retorno[1]
    A_s[4][i] = retorno[4]
    A_s[6][i] = retorno[5]
    A_s[7][i] = round((0.8*b*f_c*retorno[0]-retorno[3])/retorno[4],2)
    

tabela8 = pd.DataFrame(A_s)
tabela8 = tabela8.transpose()
tabela8.rename(columns={0:'Md', 1:'x',2:'x/d',3:'eps,s',4:'sigma,s',5:'eps,p',6:'sigma,p',7:'As'},inplace = True)
display(tabela8)

,Md,x,x/d,"eps,s","sigma,s","eps,p","sigma,p",As
0,0.0,0.51,0.005,0.64447,43.48,0.61583,148.7,-18.54
1,22582.0,3.28,0.035,0.09799,43.48,0.09815,148.7,-12.97
2,43653.0,5.92,0.062,0.05271,43.48,0.05528,148.7,-7.66
3,63119.0,8.41,0.089,0.03604,43.48,0.03949,148.7,-2.65
4,81030.0,11.04,0.116,0.02663,43.48,0.03040,148.7,-7.14
5,97419.0,13.23,0.139,0.02162,43.48,0.02568,148.7,-2.74
6,112213.0,15.26,0.161,0.01829,43.48,0.02254,148.7,1.34
7,125451.0,17.10,0.180,0.01594,43.48,0.02033,148.7,5.04
8,137157.0,18.76,0.197,0.01422,43.48,0.01871,148.7,8.38
9,147280.0,20.22,0.213,0.01294,43.48,0.01752,148.7,11.31


# Dimensionamento ao Esforço Cortante:

## Verificação da Diagonal Comprimida:

In [32]:
%%render

V_d = V_F * 1.4
tau_wd = V_d/((b)*d_sm)
tau_wu = 0.27*(1-f_ck/25)*f_ck/1.4


<IPython.core.display.Latex object>

In [33]:
if tau_wu > tau_wd:
    print("OK!")
else:
    print("NÃO OK!")

OK!


## Cálculo da Armadura de Corte:

### Armadura Mínima:

In [34]:
%%render short

rho_min = 0.2*f_ctm/f_yk
A_sws = rho_min*b

s = 15

A_sw_min = A_sws*s

nr = 2

phi_min = 2*(A_sw_min/(pi*nr))**0.5


<IPython.core.display.Latex object>

### Armadura Calculada:

In [35]:
i = int((nS+1)/2)
M_pf = prealong[1][i]
N_pf = prealong[0][i]

In [36]:
%%render

sigma_ciF = N_pf/A_c1 + M_pf/W_1
M_0 = -sigma_ciF*W_1
M_d_max = 1.4*M_F

V_c0 = 0.6*f_ctd*b*d_sm
V_c = (1+M_0/M_d_max)*V_c0

V_sw = 1.4*V_F - V_c

A_sws = V_sw/(0.9*d_sm*f_yd)

s = 20
nr = 2
phi_min = 2*(A_sws/(pi*nr))**0.5

<IPython.core.display.Latex object>

## Armadura de Introdução da Força de Protensão:

In [37]:
# VERIFICAR SE ESTA É MESMO A FORÇA DE PROTENSÃO INSERIDA NO APOIO

N_p1 = -N_p[0]

In [38]:
%%render

A_swi = (0.25*N_p1)/25.00

<IPython.core.display.Latex object>

Esta armadura deve ser distribuída num trecho equivalente à 70% do comprimento de regularização.

In [39]:
%%render

l_d = l_p*0.7

<IPython.core.display.Latex object>

# Cálculo da Flecha:

In [40]:
delta_p = []

delta_pF = 0

for i in range(len(qntCordoalhas)):
    delta_p.append([0] * 3)
    delta_p[i][0] = A_p[i]
    delta_p[i][1] = x1_anc[i]
    
    if(i != 0):
        delta_p[i][0] = delta_p[i][0] - delta_p[i-1][0]
        Momen = M_p[i] - M_p[i-1]
    else:
        Momen = M_p[i]
    
    if(delta_p[i][1] == 0):
        delta_p[i][2] = round(Momen*Vao**2/(8*E_cs*I_c1),2)
    else:
        delta_p[i][1] = (x1_anc[i]+x1[i])/2
        delta_p[i][2] = round( (Momen*(Vao**2-3*delta_p[i][1]**2-(Vao/2)**2)*(Vao/2))
                             /(6*E_cs*I_c1*Vao) ,2)
        
for i in range(len(qntCordoalhas)):
    delta_pF += delta_p[i][2]
    
tabela9 = pd.DataFrame(delta_p)
tabela9.rename(columns={0:'Ap',1:'b',2:'Delta'},inplace = True)

display(tabela9)

N_pp = N_p[len(qntCordoalhas)-1]
M_pp = M_p[len(qntCordoalhas)-1]

if (delta_p[0][0]) < 0:
    raise Exception("ERRO")

,Ap,b,Delta
0,5.72,0.000,-0.63
1,2.86,242.275,-0.13


In [41]:
%%render
delta_pF

<IPython.core.display.Latex object>

## Momento de Fissuração:


In [42]:
%%render 2

# COEFICIENTE ALPHA T:
# alpha_t = 1.2 para seções T ou duplo T
# alpha_t = 1.3 para seções I ou T invertido
# alpha_t = 1.5 para seções retangulares

alpha_t = 1.5

M_r = (alpha_t*f_ctm - N_pp/A_c1)* I_c1 / y_1 - M_pp
M_a = Q_els*0.125*Vao**2

<IPython.core.display.Latex object>

## Inércia no Estádio II:

In [43]:
%%render 2
# --------------------------------
# --------------------------------
# --------------------------------
# --------------------------------
I_II = 5.40 * 10**5 
# É NECESSÁRIO INFORMAR ESTE VALOR
# --------------------------------
# --------------------------------
# --------------------------------
# --------------------------------

<IPython.core.display.Latex object>

## Inércia Equivalente:

In [44]:
%%render

if (M_a > M_r):
    I_eq = (M_r/M_a)**3*I_c1 + (1-(M_r/M_a)**3)*I_II

if (M_a < M_r):
    I_eq = I_c1

<IPython.core.display.Latex object>

## Flecha Diferida Final:

In [45]:
%%render 2

delta_i = (delta_pF + 5/384*(Q_els*Vao**4)/(E_cs*I_eq))*(1+phi_t)

<IPython.core.display.Latex object>